<a href="https://colab.research.google.com/github/skerryvore/regression_analysis_101/blob/master/notebooks/widget_interactive_isochron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Interactive Isochron 
### To run in Colab from the Runtime menu, select Run All,  scroll down to see plot, and experiment with sliders to set slope/intercept on iscochron like, and selecting different sub-sets of data...


In [ ]:
%matplotlib inline

from ipywidgets import interactive                        # widgets and interactivity
from ipywidgets import widgets                            
from ipywidgets import Layout
from ipywidgets import Label
from ipywidgets import VBox, HBox
import matplotlib.pyplot as plt                           # plotting
import numpy as np                                        # working with arrays
import pandas as pd                                       # working with DataFrames
from scipy import stats                                   # statistical calculations


In [ ]:

# setup default font sizes for plot elements
TINY_SIZE = 12
SMALL_SIZE = 14
MEDIUM_SIZE = 20
BIGGER_SIZE = 24

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)    # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)   # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)   # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)   # legend fontsize
plt.rc('figure', titlesize=MEDIUM_SIZE)  # fontsize of the figure title
plt.rc('figure', figsize=(10, 8))        # default figure/plot size, inches

plt.rcParams['figure.figsize'] = (8, 6)

# Read csv files via link to github files online (needs internet)
phl_df = pd.read_csv('https://raw.githubusercontent.com/skerryvore/regression_analysis_101/master/data/makgenyene_phlogopite.csv')
wrk_df = pd.read_csv('https://raw.githubusercontent.com/skerryvore/regression_analysis_101/master/data/makgenyene_wholerock.csv')
cbn_df = pd.read_csv('https://raw.githubusercontent.com/skerryvore/regression_analysis_101/master/data/makgenyene_carbonate.csv')
all_df = pd.concat([phl_df,wrk_df,cbn_df])


In [ ]:
phl_style = {'marker':'o','s':50,'color':'k','label':'phlogopites'}
wrk_style = {'marker':'s','s':50,'color':'r','label':'whole rock'}
cbn_style = {'marker':'d','s':50,'color':'g','label':'carbonates'}

# interactive calculation of the sample set (control of source parametric distribution and number of samples)
l = widgets.Text(value='Interactive Isochron, University of Glasgow',layout=Layout(width='950px', height='30px'))

samples = widgets.Dropdown(
    options=['Phlogopite', 'Wholerock', 'Carbonate', 'All'],
    description='Sample type:',
    disabled=False,
    layout=Layout(width='200px', height='30px')
)

slope = widgets.FloatSlider(min=0.0, max = 0.004, value = 0.0, step = 0.0001, description = 'Slope:',\
        orientation='horizontal',readout_format='7.5f',layout=Layout(width='400px', height='50px'))
slope.style.handle_color = 'blue'

intercept = widgets.FloatSlider(min=0.6, max = 1.2, value = 0.7, step = 0.01, description = 'Intercept:',\
        orientation='horizontal',readout_format='6.3f',layout=Layout(width='400px', height='50px'))
intercept.style.handle_color = 'green'


ui = widgets.VBox([samples,slope,intercept],)       # basic widget formatting   
ui2 = widgets.VBox([l,ui],)

def plot_isochron(samples,slope,intercept): # function to take parameters, make sample and plot
    
    df, style = select_data(samples)
    
    x = np.linspace(0,300,10)
    ypred = np.array(df['87Rb/86Sr'])*slope + intercept
    ydata = np.array(df['87Sr/86Sr'])
    ymean = np.mean(ydata)
    
    # Calculate the age in My using the slope
    lamda87Rb = 1.419E-11 # decay constant for 87Rb in 1/yr

    age = (np.log(slope +1)/lamda87Rb)/1.E6 # Divide by 1.E6 to convert to Ma

    # Calculate RMS error
    rmse = np.sqrt(np.sum((ypred - ydata)**2))

    sse = np.sum((ypred - ydata)**2)
    ssr = np.sum((ypred-ymean)**2)
    sst = sse + ssr
    r_squared = ssr/sst
    age_err = age*np.sqrt(sse)

    print(f'RMSE: {rmse:04.2f}  [{87}Sr/{86}Sr] \n')
    print(f'r_squaed: {r_squared:04.3f}\n')
    print(f'Isochron Age: {age:06.2f} +/- {age_err:3.2f} Ma \n')
    
    # Plot isochron diagram, 87Sr/86Sr on y-axis versus 87Rb/86Sr on the x-axis

    fig2, ax1 = plt.subplots() 

    ax1.set_title("Isochron diagram\n")
    ax1.set_ylabel("$^{87}Sr\,/\,^{86}Sr$")
    ax1.set_xlabel("$^{87}Rb\,/\,^{86}Sr$")
    
    if (samples == 'All'):
      ax1.scatter(np.array(phl_df['87Rb/86Sr']), np.array(phl_df['87Sr/86Sr']), **phl_style)
      ax1.scatter(np.array(wrk_df['87Rb/86Sr']), np.array(wrk_df['87Sr/86Sr']), **wrk_style)
      ax1.scatter(np.array(cbn_df['87Rb/86Sr']), np.array(cbn_df['87Sr/86Sr']), **cbn_style)
      #Plot interactive isochron line
      ax1.plot(x, (x*slope+intercept),'r-', linewidth=2, markersize=0)
    
    elif (samples == 'Phlogopite'):
      ax1.scatter(np.array(phl_df['87Rb/86Sr']), np.array(phl_df['87Sr/86Sr']), **phl_style)
      ax1.scatter(np.array(wrk_df['87Rb/86Sr']), np.array(wrk_df['87Sr/86Sr']), alpha=0.2,**wrk_style)
      ax1.scatter(np.array(cbn_df['87Rb/86Sr']), np.array(cbn_df['87Sr/86Sr']), alpha=0.2,**cbn_style)
      #Plot interactive isochron line
      ax1.plot(x, (x*slope+intercept),'r-', linewidth=2, markersize=0)
     
    elif (samples == 'Wholerock'):
      ax1.scatter(np.array(phl_df['87Rb/86Sr']), np.array(phl_df['87Sr/86Sr']), alpha=0.2,**phl_style)
      ax1.scatter(np.array(wrk_df['87Rb/86Sr']), np.array(wrk_df['87Sr/86Sr']), **wrk_style)
      ax1.scatter(np.array(cbn_df['87Rb/86Sr']), np.array(cbn_df['87Sr/86Sr']), alpha=0.2,**cbn_style)
     #Plot interactive isochron line
      ax1.plot(x, (x*slope+intercept),'r-', linewidth=2, markersize=0)
      ax1.set_xlim([0.0, 6.0])
      ax1.set_ylim([0.69, 0.74])
        
    elif (samples == 'Carbonate'):
      ax1.scatter(np.array(phl_df['87Rb/86Sr']), np.array(phl_df['87Sr/86Sr']), alpha=0.2,**phl_style)
      ax1.scatter(np.array(wrk_df['87Rb/86Sr']), np.array(wrk_df['87Sr/86Sr']), alpha=0.2,**wrk_style)
      ax1.scatter(np.array(cbn_df['87Rb/86Sr']), np.array(cbn_df['87Sr/86Sr']), **cbn_style)
    #Plot interactive isochron line
      ax1.plot(x, (x*slope+intercept),'r-', linewidth=2, markersize=0)
      ax1.set_xlim([0.0, 6.0])
      ax1.set_ylim([0.69, 0.74])
    
    # Draw the legend for the plot
    ax1.legend(loc='lower right')
    plt.legend()
    
    plt.show()

def select_data(samples):            # function to select samples to plot (pick dataframe) 
    
    if samples == 'Phlogopite':
        df = phl_df
        style = phl_style
        return df,style
    
    elif samples == 'Wholerock':
        df = wrk_df
        style = wrk_style
        return df,style
    
    elif samples == 'Carbonate':
        df= cbn_df
        style = cbn_style
        return df,style
    
    elif samples == 'All':
        df= all_df
        style = {'marker':'o','s':50,'color':'b','label':'all samples'}
        return df,style

# connect the function to make the samples and plot to the widgets    
interactive_plot = widgets.interactive_output(plot_isochron, {'samples': samples,'slope': slope, 'intercept': intercept})
interactive_plot.clear_output(wait = True)               # reduce flickering by delaying plot updating

# Print top 20 rows of phlogopite data table    
all_df.head(30)


,Sample,Rb [ppm],Rb_error,Sr [ppm],Sr_error,87Rb/86Sr,Rb/Sr_error,87Sr/86Sr,2_sigma,1_sigma
0,63-B2,511.00,4.000,35.70,0.5,41.800,0.290,0.781250,0.00024,0.000120
1,63-B3,463.00,4.000,29.80,0.4,45.400,0.320,0.787860,0.00060,0.000300
2,64-B1,485.00,4.000,911.00,13.7,1.540,0.010,0.712565,0.00007,0.000035
3,64-B2,556.00,4.000,43.90,0.7,37.000,0.260,0.772660,0.00060,0.000300
4,64-B3,497.00,4.000,15.30,0.2,96.000,0.670,0.897830,0.00084,0.000420
5,65-B1,492.00,4.000,1421.00,21.3,1.000,0.010,0.711700,0.00005,0.000025
6,65-B2a,504.00,4.000,188.00,2.8,7.760,0.050,0.723560,0.00012,0.000060
7,65-B2b,521.00,4.000,146.00,2.2,10.400,0.070,0.727800,0.00010,0.000050
8,65-B3,460.00,4.000,30.90,0.5,43.200,0.300,0.783750,0.00066,0.000330
9,66-G1,663.00,5.000,35.30,0.5,55.000,0.390,0.803320,0.00015,0.000075


In [ ]:

display(ui2, interactive_plot)                            # display the interactive plot

Output()